# 피봇 테이블
### 피봇 테이블(pivot talbe) : 데이터 열(column) 중에서 두 개를 키(key)로 사용하여 데이터를 선택하는 방법

#### 키가 될 수 있는 두 개의 열 혹은 필드를 선택 -> 두 열을 행인덱스, 열인덱스로 변경
#### pivot 메서드 : 첫 번쨰 인수=행 인덱스로 사용할 열이름, 두 번쨰 인수=열인덱스로 사용할 열이름, 마지막=데이터로 사용할 열 이름

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
data = {'state': ['Ohio','Ohion','Ohion','Nevada','Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002],
        'pop': [1.5, 2.5, 3.0, 2.5, 3.5]
}
df = pd.DataFrame(data, columns=["state", "year", "pop"])
df

,state,year,pop
0,Ohio,2000,1.5
1,Ohion,2001,2.5
2,Ohion,2002,3.0
3,Nevada,2001,2.5
4,Nevada,2002,3.5


In [3]:
df.pivot("state", "year", "pop")

year,2000,2001,2002
state,,,
Nevada,NaN,2.5,3.5
Ohio,1.5,NaN,NaN
Ohion,NaN,2.5,3.0


In [4]:
df.set_index(["state", "year"]).unstack()

pop          
year   2000 2001 2002
state                
Nevada  NaN  2.5  3.5
Ohio    1.5  NaN  NaN
Ohion   NaN  2.5  3.0

### * 행 인덱스와, 열 인덱스가 될 자료는 키(key)의 역할을 해야 한다. 즉, 이 값으로 데이터가 유일하게(unique) 결정되어야 한다. 만약 조건에 해당하는 데이터가 2개 이상인 경우에는 에러가 발생한다.

In [5]:
df.pivot("year","pop","state")

ValueError: Index contains duplicate entries, cannot reshape

# * 그룹 분석
### 데이터가 복수개 있어도 ok(피봇 테이블과 차이) -> 연산을 통해 복수개의 그룹 데이터에 대한 대표값 설정
### - groupby 명령 사용
### - 분석하고자 하는 시리즈나 데이터프레임에 groupby 메서드 호출
### - 호출한 결과에 그룹 연산을 수행

## * groupby 메서드 : 데이터를 그룹 별로 분류하는 역할. 
### 열 또는 열의 리스트 / 행 인덱스 -> 인수로 사용
### 연산 결과로 GroupBy 클래스 객체를 반환 -> 이 객체에는 그룹 별로 연산을 할 수 있는 그룹 연산 메서드 존재

## * 그룹 연산 메서드
### - size(), count() : 갯수
### - mean(), median(), min(), max()
### - sum(),prod(), std(), quantile()
### - first(), last()
### - agg(), aggregate()
### - transform() : 그룹 연산으로 대표값을 만든 다음 이 대표 값을 새로운 열로 원래 데이터프레임에 추가
### - describe() : 하나의 그룹 대표값이 아니라 여러개의 값을 데이터 프레임으로 구한다
### apply() : describe()처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹 연산이 없는 경우에 사용

In [5]:
np.random.seed(0)
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.764052,-0.977278,a,one
1,0.400157,0.950088,a,two
2,0.978738,-0.151357,b,one
3,2.240893,-0.103219,b,two
4,1.867558,0.410599,a,one


In [6]:
df.groupby(df.key1).mean()

,data1,data2
key1,,
a,1.343923,0.127803
b,1.609816,-0.127288


In [7]:
df.groupby(df.key1)["data1"].mean()

key1
a    1.343923
b    1.609816
Name: data1, dtype: float64

In [8]:
df.groupby(df.key1).mean()["data1"]

key1
a    1.343923
b    1.609816
Name: data1, dtype: float64

In [9]:
df.data1.groupby([df.key1, df.key2]).mean()

key1  key2
a     one     1.815805
      two     0.400157
b     one     0.978738
      two     2.240893
Name: data1, dtype: float64

In [10]:
df.data1.groupby([df.key1, df.key2]).mean().unstack("key2")

key2,one,two
key1,,
a,1.815805,0.400157
b,0.978738,2.240893


# pivot_table
### pandas
#### pivot_table : pivot명령과 groupby 명령의 중간적 성격
#### - groupby 명령처럼 그룹 분석 -> 최종적으로는 pivot명령처럼 피봇 테이블 생성
#### - groupby 명령의 결과에 unstack을 자동적용하여 2차원적 형태로 변형
### pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')
#### - data: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)
#### - values: 분석할 데이터프레임에서 분석할 열
#### - index: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트
#### - columns: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트
#### - aggfunc: 분석 메서드
#### - fill_value: NaN 대체 값
#### - margins: 오른쪽과 아래에 합계를 붙일지 여부
#### - margins_name: 합계 열(행)의 이름
#### => 일반 피봇테이블의 관점 : pivot을 수행하지만 데이터가 유니크하게 선택도지 않으면 aggfunc 인수로 정의된 함수를 수행하여 대표값 계산

In [11]:
df.pivot_table("data1", "key1", "key2")

key2,one,two
key1,,
a,1.815805,0.400157
b,0.978738,2.240893


In [12]:
df.pivot_table("data1", "key1", "key2", margins=True, margins_name="합계")

key2,one,two,합계
key1,,,
a,1.815805,0.400157,1.343923
b,0.978738,2.240893,1.609816
합계,1.536783,1.320525,1.450280


# Tip 데이터 예제
### 컬럼 설명
#### - total_bill: 식사대금
#### - tip: 팁
#### - sex: 성별
#### - smoker: 흡연/금연 여부
#### - day: 요일
#### - time: 시간
#### - size: 인원

In [15]:
tips = sns.load_dataset("tips")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [16]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


In [17]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


# 그룹별 통계

In [19]:
tips.groupby("sex").count() # NaN이 있을 경우 영향 있음

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [21]:
tips.groupby("sex").size() # Nan이 있어도 상관 없음

sex
Male      157
Female     87
dtype: int64

In [22]:
tips.groupby(["sex", "smoker"]).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [24]:
tips.pivot_table("tip_pct","sex","smoker",aggfunc="count",margins=True)

smoker,Yes,No,All
sex,,,
Male,60.0,97.0,157.0
Female,33.0,54.0,87.0
All,93.0,151.0,244.0


In [25]:
tips.pivot_table("tip_pct","sex","smoker")

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


In [27]:
tips.groupby(["sex", "smoker"])[["tip","tip_pct"]].describe()

tip    tip_pct
sex    smoker                            
Male   Yes    count  60.000000  60.000000
              mean    3.051167   0.152771
              std     1.500120   0.090588
              min     1.000000   0.035638
              25%     2.000000   0.101845
              50%     3.000000   0.141015
              75%     3.820000   0.191697
              max    10.000000   0.710345
       No     count  97.000000  97.000000
              mean    3.113402   0.160669
              std     1.489559   0.041849
              min     1.250000   0.071804
              25%     2.000000   0.131810
              50%     2.740000   0.157604
              75%     3.710000   0.186220
              max     9.000000   0.291990
Female Yes    count  33.000000  33.000000
              mean    2.931515   0.182150
              std     1.219916   0.071595
              min     1.000000   0.056433
              25%     2.000000   0.152439
              50%     2.880000   0.173913
              75%     3.500000   0.198216
              max     6.500000   0.416667
       No     count  54.000000  54.000000
              mean    2.773519   0.156921
              std     1.128425   0.036421
              min     1.000000   0.056797
              25%     2.000000   0.139708
              50%     2.680000   0.149691
              75%     3.437500   0.181630
              max     5.200000   0.252672

In [29]:
def peak_to_peak(x):
    return x.max() - x.min()

tips.groupby(["sex","smoker"])[["tip"]].agg(peak_to_peak)

tip
sex    smoker      
Male   Yes     9.00
       No      7.75
Female Yes     5.50
       No      4.20

In [30]:
tips.groupby(["sex","smoker"]).agg(["mean", peak_to_peak])[["total_bill"]]

total_bill             
                    mean peak_to_peak
sex    smoker                        
Male   Yes     22.284500        43.56
       No      19.791237        40.82
Female Yes     17.977879        41.23
       No      18.105185        28.58

In [31]:
tips.groupby(["sex", "smoker"]).agg({'tip_pct' : 'mean', 'total_bill' : peak_to_peak})

tip_pct  total_bill
sex    smoker                      
Male   Yes     0.152771       43.56
       No      0.160669       40.82
Female Yes     0.182150       41.23
       No      0.156921       28.58

In [32]:
tips.pivot_table(index=['sex', 'smoker'])

size       tip   tip_pct  total_bill
sex    smoker                                          
Male   Yes     2.500000  3.051167  0.152771   22.284500
       No      2.711340  3.113402  0.160669   19.791237
Female Yes     2.242424  2.931515  0.182150   17.977879
       No      2.592593  2.773519  0.156921   18.105185

In [33]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'], columns='smoker')

tip_pct                size          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  0.164417  0.165706  2.300000  2.500000
       Fri   0.144730  0.138005  2.125000  2.000000
       Sat   0.139067  0.162132  2.629630  2.656250
       Sun   0.173964  0.158291  2.600000  2.883721
Female Thur  0.163073  0.155971  2.428571  2.480000
       Fri   0.209129  0.165296  2.000000  2.500000
       Sat   0.163817  0.147993  2.200000  2.307692
       Sun   0.237075  0.165710  2.500000  3.071429

In [34]:
tips.pivot_table('size', index=['time', 'sex', 'smoker'], columns='day', aggfunc='sum', fill_value=0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43

# 연습문제1
#### 타이타닉 승객 데이터를 이용하여 다음 분석을 실시하라.
### titanic = sns.load_dataset("titanic")
#### 1. qcut 명령으로 세 개의 나이 그룹을 만든다.
#### 2. 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다.
#### 3. 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.

In [39]:
titanic = sns.load_dataset("titanic")
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True


In [59]:
titanic['agen'] = pd.qcut(titanic['age'], 3, labels=["1", "2", "3"])
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,agen
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,2
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,1
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,NaN
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,2
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,2


In [60]:
titanic.pivot_table('survived', index=['sex', 'agen'], aggfunc='mean', columns='class')

class           First    Second     Third
sex    agen                              
female 1     0.954545  1.000000  0.508475
       2     0.947368  0.909091  0.481481
       3     0.977273  0.857143  0.250000
male   1     0.500000  0.357143  0.158879
       2     0.500000  0.076923  0.195652
       3     0.347826  0.062500  0.055556

In [61]:
titanic.pivot_table("survived", "sex", "class")

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447
